# House price prediction
Firstly, we'd better to analyze the data, we'll do from the four aspects:
1. Drop the useless columns
   1. Columns have too many Nan values
   2. Less correlate with house price column
2. Substitute the Nan value of useful columns

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
import seaborn as sns

data_dictionary.txt
macro.csv
sample_submission.csv
test
test.csv
train
train.csv



In [2]:
train_dataset_org = pd.read_csv("../input/train.csv")
train_dataset_org.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [8]:
# As there are so many Nan, so let's watch the data without Nan to identify the value of them
train_dataset_org.loc[:,train_dataset_org.isnull().sum() == 0].head()

,id,timestamp,full_sq,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,Investment,Bibirevo,6.407578e+06,155572,0.189727,0.000070,9576,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,Investment,Nagatinskij Zaton,9.589337e+06,115352,0.372602,0.049637,6880,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,Investment,Tekstil'shhiki,4.808270e+06,101708,0.112560,0.118537,5879,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,Investment,Mitino,1.258354e+07,178473,0.194703,0.069753,13087,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,Investment,Basmannoe,8.398461e+06,108171,0.015234,0.037316,5706,...,319,108,17,135,236,2,91,195,14,16331452


In [9]:
# Show the shape of dataset
print(train_dataset_org.shape)

(30471, 292)


In [10]:
null_count = train_dataset_org.isnull().sum().sort_values(ascending = False)
null_percent = ( train_dataset_org.isnull().sum() / train_dataset_org.isnull().count()).sort_values(ascending = False)
null_info = pd.concat([null_count, null_percent], axis = 1, keys = ['count', 'percent'])
print(null_info)

                                       count   percent
hospital_beds_raion                    14441  0.473926
build_year                             13605  0.446490
state                                  13559  0.444980
cafe_avg_price_500                     13281  0.435857
cafe_sum_500_max_price_avg             13281  0.435857
cafe_sum_500_min_price_avg             13281  0.435857
max_floor                               9572  0.314135
material                                9572  0.314135
num_room                                9572  0.314135
kitch_sq                                9572  0.314135
preschool_quota                         6688  0.219487
school_quota                            6685  0.219389
cafe_sum_1000_min_price_avg             6524  0.214105
cafe_sum_1000_max_price_avg             6524  0.214105
cafe_avg_price_1000                     6524  0.214105
life_sq                                 6383  0.209478
build_count_frame                       4991  0.163795
build_coun

In [11]:
# check columns of dataset
print(len(train_dataset_org.columns.values))
print("=" * 50)
print(train_dataset_org.columns.values)

292
['id' 'timestamp' 'full_sq' 'life_sq' 'floor' 'max_floor' 'material'
 'build_year' 'num_room' 'kitch_sq' 'state' 'product_type' 'sub_area'
 'area_m' 'raion_popul' 'green_zone_part' 'indust_part'
 'children_preschool' 'preschool_quota' 'preschool_education_centers_raion'
 'children_school' 'school_quota' 'school_education_centers_raion'
 'school_education_centers_top_20_raion' 'hospital_beds_raion'
 'healthcare_centers_raion' 'university_top_20_raion' 'sport_objects_raion'
 'additional_education_raion' 'culture_objects_top_25'
 'culture_objects_top_25_raion' 'shopping_centers_raion' 'office_raion'
 'thermal_power_plant_raion' 'incineration_raion' 'oil_chemistry_raion'
 'radiation_raion' 'railroad_terminal_raion' 'big_market_raion'
 'nuclear_reactor_raion' 'detention_facility_raion' 'full_all' 'male_f'
 'female_f' 'young_all' 'young_male' 'young_female' 'work_all' 'work_male'
 'work_female' 'ekder_all' 'ekder_male' 'ekder_female' '0_6_all' '0_6_male'
 '0_6_female' '7_14_all' '7_14_ma

In [12]:
# Let's look at the types of all columns values
train_dataset_org.dtypes.loc[:].unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [27]:
# show all object columns
train_dataset_org.dtypes.loc[train_dataset_org.dtypes == np.object]

timestamp                    object
product_type                 object
sub_area                     object
culture_objects_top_25       object
thermal_power_plant_raion    object
incineration_raion           object
oil_chemistry_raion          object
radiation_raion              object
railroad_terminal_raion      object
big_market_raion             object
nuclear_reactor_raion        object
detention_facility_raion     object
water_1line                  object
big_road1_1line              object
railroad_1line               object
ecology                      object
dtype: object

In [30]:
# show all float64 columnes
train_dataset_org.dtypes.loc[train_dataset_org.dtypes == np.float64]

life_sq                                  float64
floor                                    float64
max_floor                                float64
material                                 float64
build_year                               float64
num_room                                 float64
kitch_sq                                 float64
state                                    float64
area_m                                   float64
green_zone_part                          float64
indust_part                              float64
preschool_quota                          float64
school_quota                             float64
hospital_beds_raion                      float64
raion_build_count_with_material_info     float64
build_count_block                        float64
build_count_wood                         float64
build_count_frame                        float64
build_count_brick                        float64
build_count_monolith                     float64
build_count_panel   

In [28]:
# show all int64 columns
train_dataset_org.dtypes.loc[train_dataset_org.dtypes == np.int64]

id                                       int64
full_sq                                  int64
raion_popul                              int64
children_preschool                       int64
preschool_education_centers_raion        int64
children_school                          int64
school_education_centers_raion           int64
school_education_centers_top_20_raion    int64
healthcare_centers_raion                 int64
university_top_20_raion                  int64
sport_objects_raion                      int64
additional_education_raion               int64
culture_objects_top_25_raion             int64
shopping_centers_raion                   int64
office_raion                             int64
full_all                                 int64
male_f                                   int64
female_f                                 int64
young_all                                int64
young_male                               int64
young_female                             int64
work_all     

As shown above, there are three data types for all columns, let's list all columns of each data type:

In [14]:
train_dataset_x_timestamp = train_dataset_org.drop(["id",timestamp"], axis = 1)
train_dataset_x_timestamp.head()

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [13]:
# Data type is int64
train_dataset_org.loc[:, train_dataset_org.dtypes == np.int64].describe()

,id,full_sq,raion_popul,children_preschool,preschool_education_centers_raion,children_school,school_education_centers_raion,school_education_centers_top_20_raion,healthcare_centers_raion,university_top_20_raion,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
count,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,...,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,30471.000000,3.047100e+04
mean,15237.917397,54.214269,84056.425552,5140.026156,4.065111,5354.269699,4.705031,0.109678,1.320895,0.138295,...,32.058318,10.783860,1.771783,15.045552,30.251518,0.442421,8.648814,52.796593,5.987070,7.123035e+06
std,8796.501536,38.031487,57871.285899,3816.625140,2.993795,3989.640917,3.445105,0.333328,1.492903,0.443796,...,73.465611,28.385679,5.418807,29.118668,47.347938,0.609269,20.580741,46.292660,4.889219,4.780111e+06
min,1.000000,0.000000,2546.000000,175.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+05
25%,7620.500000,38.000000,21819.000000,1706.000000,2.000000,1564.000000,2.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,0.000000,2.000000,9.000000,0.000000,0.000000,11.000000,1.000000,4.740002e+06
50%,15238.000000,49.000000,83502.000000,4857.000000,4.000000,5261.000000,5.000000,0.000000,1.000000,0.000000,...,8.000000,2.000000,0.000000,7.000000,16.000000,0.000000,2.000000,48.000000,5.000000,6.274411e+06
75%,22855.500000,63.000000,122862.000000,7103.000000,6.000000,7227.000000,7.000000,0.000000,2.000000,0.000000,...,21.000000,5.000000,1.000000,12.000000,28.000000,1.000000,7.000000,76.000000,10.000000,8.300000e+06
max,30473.000000,5326.000000,247469.000000,19223.000000,13.000000,19083.000000,14.000000,2.000000,6.000000,3.000000,...,377.000000,147.000000,30.000000,151.000000,250.000000,2.000000,106.000000,218.000000,21.000000,1.111111e+08


In [17]:
# get all object columns
train_dataset_x_timestamp.loc[:,train_dataset_x_timestamp.dtypes == np.object].head()

,product_type,sub_area,culture_objects_top_25,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,water_1line,big_road1_1line,railroad_1line,ecology
0,Investment,Bibirevo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,Investment,Nagatinskij Zaton,yes,no,no,no,no,no,no,no,no,no,no,no,excellent
2,Investment,Tekstil'shhiki,no,no,no,no,yes,no,no,no,no,no,no,no,poor
3,Investment,Mitino,no,no,no,no,no,no,no,no,no,no,no,no,good
4,Investment,Basmannoe,no,no,no,no,yes,yes,no,no,no,no,no,yes,excellent
